## 第8章　数値型

In [1]:
import numpy as np
import pandas as pd

### 8-1　数値型への変換

In [2]:
print(type(40000 / 3))

print(type(int(40000 / 3)))

print(type(float(40000 / 3)))

<class 'float'>
<class 'int'>
<class 'float'>


In [3]:
df = pd.DataFrame({'value': [40000 / 3]})

print(df.dtypes)
df.head()

value    float64
dtype: object


,value
0,13333.333333


In [4]:
df['value'].astype('int8')
df['value'].astype('int16')
df['value'].astype('int32')
df['value'].astype('int64')

0    13333
Name: value, dtype: int64

In [5]:
df['value'].astype('float16')
df['value'].astype('float32')
df['value'].astype('float64')

0    13333.333333
Name: value, dtype: float64

大まかなデータ型指定の場合、クォート（' '）は不要となる。

In [6]:
df['value'].astype(int)
df['value'].astype(float)

0    13333.333333
Name: value, dtype: float64

### 8-2　対数化による非線形な変化

対数化とは、値が大きくなるほど、値の差の意味を小さくしたいときに有効な手法である。<br>
例えば、10から11歳への身長の伸びと、50から51歳への身長の伸びは異なる場合などに必要となる。

In [7]:
reserve_tb = pd.read_csv('reserve.csv')

reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [8]:
reserve_tb['total_price_log'] = reserve_tb['total_price'].apply(lambda x: np.log10(x / 1000 + 1))

reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,total_price_log
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.992111
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,1.334454
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,1.539076
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,2.290925
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,1.839478


上の表を見ての通り、値が大きいものほど、なるべく小さく対数変換されていることが分かる。

### 8-3　カテゴリ化による非線形な変化

In [12]:
customer_tb = pd.read_csv('customer.csv')

customer_tb.head()

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797


In [13]:
customer_tb['age_rank'] = (np.floor(customer_tb['age'] / 10) * 10).astype('category')

print(customer_tb.dtypes)
print(customer_tb.shape)
customer_tb.head()

customer_id         object
age                  int64
sex                 object
home_latitude      float64
home_longitude     float64
age_rank          category
dtype: object
(1000, 6)


,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40.0
1,c_2,38,man,35.325076,139.410551,30.0
2,c_3,49,woman,35.120543,136.511179,40.0
3,c_4,43,man,43.034868,141.240314,40.0
4,c_5,31,man,35.102661,136.523797,30.0


np.floor関数は、引数の小数点以下を切り捨てることができる。（正確には、引数以下の最大整数を返す）

そのため、1行目の41歳であれば、41を10で割って4.1する。<br>
小数点を切り捨てして4.0にしてから、10倍して40にすることで、rank値を与えることができる。